In [1]:
# Import library yang dibutuhkan
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import json

from sklearn.model_selection import StratifiedKFold

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1" # Kalau GPU nya cuma 1 ini gak perlu di panggil

In [3]:
# List dataset
train_dataset_path = "../datasets/coffee/train"
list_katagori = sorted(os.listdir(train_dataset_path))

list_dataset_training = []
for kategori in sorted(list_katagori):
    path_kategori = os.path.join(train_dataset_path, kategori)
    list_dataset = [
        {'filename': os.path.join(path_kategori, nama_file), "class": kategori}
        for nama_file in sorted(os.listdir(path_kategori))
        if nama_file.endswith(".jpg")
    ]
    list_dataset_training.extend(list_dataset)

dataframe_dataset_training = pd.DataFrame.from_dict(list_dataset_training)
dataframe_dataset_training.head()

,filename,class
0,../datasets/coffee/train/defect/0.jpg,defect
1,../datasets/coffee/train/defect/1.jpg,defect
2,../datasets/coffee/train/defect/10.jpg,defect
3,../datasets/coffee/train/defect/100.jpg,defect
4,../datasets/coffee/train/defect/101.jpg,defect


In [4]:
# Lihat distribusi dataset
print(dataframe_dataset_training.groupby('class').size())

# Lihat jumlah kategori
jumlah_kategori = len(dataframe_dataset_training.groupby('class').size())
print(f"Jumlah kategori: {jumlah_kategori} -> {list_katagori}")

class
defect       300
longberry    300
peaberry     300
premium      300
dtype: int64
Jumlah kategori: 4 -> ['defect', 'longberry', 'peaberry', 'premium']


In [5]:
# Menginisialisasi StratifiedKFold untuk membagi dataset dan mencari yang terbaik
# Ref: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=333)

In [6]:
# Menginisialisasi Keras ImageDataGenerator untuk memuat, memproses, dan mengaukmentasi gambar
# Ref: https://keras.io/api/data_loading/image/

augmentation_setting = {
    "rotation_range":90,
    "width_shift_range":0.15,
    "height_shift_range":0.15,
    "brightness_range":(0.85, 1.25),
    "shear_range":0.15,
    "zoom_range":0.2,
    "horizontal_flip":True,
    "vertical_flip":True
}

# Train set di augmentasi agar dataset lebih banyak dan modelnya bisa lebih robust
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.15,
    **augmentation_setting
)

# Test set tidak perlu di augmentasi karena ketika testing gambar asli yang di gunakan 
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255.0,
)

In [7]:
# Memuat Arsitektur CNN yang akan dipakai. Berdasarkan experiment, kita memilih untuk mengunakan DenseNet
# Paper DenseNet: https://arxiv.org/abs/1608.06993
from tensorflow.keras.applications import DenseNet169

In [8]:
# Set Parameter Training
batch_size = 32
ukuran_input = (224, 224)
epochs = 150

In [ ]:
hasil_evaluasi_model = []

# Membagi dataset dengan StratifiedKFold
for index_fold, (index_training, index_test) in enumerate(
    skf.split(dataframe_dataset_training, dataframe_dataset_training['class'])
):

    print(f'Memulai potongan dataset ke-{index_fold}')

    # Mengambil potongan dataset
    data_training = dataframe_dataset_training.iloc[index_training]
    data_test = dataframe_dataset_training.iloc[index_test]

    # Inisialisasi dataset generator untuk setiap subset
    train_data_generator = train_generator.flow_from_dataframe(
        data_training,
        x_col='filename',
        y_col='class',
        class_mode='categorical',
        classes=list_katagori,
        batch_size=batch_size,
        target_size=ukuran_input,
        shuffle=True,
    )

    test_data_generator = test_generator.flow_from_dataframe(
        data_test,
        x_col='filename',
        y_col='class',
        class_mode='categorical',
        classes=list_katagori,
        target_size=ukuran_input,
        shuffle=False,
    )
    
    all_data_generator =test_generator.flow_from_dataframe(
        dataframe_dataset_training,
        x_col='filename',
        y_col='class',
        class_mode='categorical',
        classes=list_katagori,
        target_size=ukuran_input,
        shuffle=False,
    )

    # Inisalisasi model deep learning
    tf.keras.backend.clear_session()

    model = DenseNet169(
        include_top=True,
        weights=None,
        classes=jumlah_kategori,
        classifier_activation="softmax"
    )

    model.compile(
        optimizer="adam",
        loss='categorical_crossentropy',
        metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
    )

    # Print arsitektur model
#         model.summary()

    # Inisialisasi wandb untuk tracking experiments
    model_name = f"Coffee_DenseNet169_raw-fold_{index_fold}"

    # Inisialisasi callbacks

    checkpoint_path = f'../models/{model_name}-best_val.ckpt'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, mode='min', save_best_only=True, save_weights_only=True
    )
    reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
        mode='min', factor=0.1, patience=20)
    
    all_callbacks = [model_checkpoint_callback, reduce_lr_callback]

    # Training model
    history = model.fit(
        train_data_generator,
        validation_data=test_data_generator,
        epochs=epochs,
        callbacks=all_callbacks
    )

    # Load the best train model
    model.save(f"../models/{model_name}-best_train.keras")

    # Evaluate the splited testset
    eval_results = {}

    # Evaluate the best train model
    for data_name, eval_data_gen in zip(
        ["train", "test", "all"], [train_data_generator, test_data_generator, all_data_generator]
    ):
        
        print(f"Evaluating {data_name} Set...")
        
        eval_data = model.evaluate(eval_data_gen)

        for metric_name, eval_result in zip(model.metrics_names, eval_data):
            eval_results[f"best_train/{data_name}/{metric_name}"] = float(eval_result)
   

    # Evaluate the best val model
    model.load_weights(checkpoint_path)
    model.save(f"../models/{model_name}-best_test.keras")
    
    # Evaluate the best train model
    for data_name, eval_data_gen in zip(
        ["train", "test", "all"], [train_data_generator, test_data_generator, all_data_generator]
    ):
        
        print(f"Evaluating {data_name} Set...")
        
        eval_data = model.evaluate(eval_data_gen)

        for metric_name, eval_result in zip(model.metrics_names, eval_data):
            eval_results[f"best_test/{data_name}/{metric_name}"] = float(eval_result)
            
    hasil_evaluasi_model.append(
        {model_name: eval_results}
    )        
    
# Save the evaluation metrics to JSON
with open("hasil_evaluasi.json", "w") as outfile:
    json.dump(hasil_evaluasi_model, outfile)

Memulai potongan dataset ke-0
Found 800 validated image filenames belonging to 4 classes.
Found 400 validated image filenames belonging to 4 classes.
Found 1200 validated image filenames belonging to 4 classes.
Epoch 1/150
25/25 [==============================] - 41s 777ms/step - loss: 1.3416 - accuracy: 0.4575 - precision: 0.5280 - recall: 0.2825 - val_loss: 1.5876 - val_accuracy: 0.2550 - val_precision: 0.1290 - val_recall: 0.0100 - lr: 0.0010
Epoch 2/150
25/25 [==============================] - 13s 521ms/step - loss: 1.0783 - accuracy: 0.5375 - precision: 0.6385 - recall: 0.3775 - val_loss: 1.8864 - val_accuracy: 0.2500 - val_precision: 0.2506 - val_recall: 0.2500 - lr: 0.0010
Epoch 3/150
25/25 [==============================] - 13s 513ms/step - loss: 0.9898 - accuracy: 0.6175 - precision: 0.6868 - recall: 0.4825 - val_loss: 2.3745 - val_accuracy: 0.2500 - val_precision: 0.2500 - val_recall: 0.2500 - lr: 0.0010
Epoch 4/150
25/25 [==============================] - 13s 515ms/step - lo